In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import plotly.express as px
import matplotlib.pyplot as plt
import warnings

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, Ridge, SGDRegressor
from sklearn.svm import LinearSVR
from sklearn.preprocessing import StandardScaler, OneHotEncoder, KBinsDiscretizer
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import make_column_transformer, TransformedTargetRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA


from sklearn.inspection import permutation_importance

from sklearn.ensemble import BaggingRegressor, RandomForestRegressor
from xgboost import XGBRFRegressor, XGBRegressor


plt.style.use("seaborn-whitegrid")

pd.set_option("display.max_columns", None)

warnings.filterwarnings("ignore")

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import time

from yellowbrick.regressor import ResidualsPlot
from yellowbrick.model_selection import LearningCurve

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        


# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
print("Ok")

In [ ]:
def _get_model_name(model):
    """
        Returns a string with the name of a sklearn model
            model: Sklearn stimator class
    """
    if isinstance(model, Pipeline):
        estimator = model.steps[-1][1]
        name = "Pipeline_" + str(estimator)[:str(estimator).find("(")]
    else: 
        name = str(model)[:str(model).find("(")]
    return name 

def plot_cv_score(X, y, models_list, cv = 5, scoring = None, refit = True, verbose = True):
    """ 
        X: numpy_array/pandas dataframe n_rows, m_features
        y: numpy_array/pandas dataframe n_rows
        Plots min, max and avg kfold crosval_score for a list of models
    
    """

    
    
    names, scores, min_score, max_score, mean_score = list(), list(), list(), list(), list()

    for i, model in enumerate(models_list):
        t0 = time.time()
        name = _get_model_name(model)
        names.append(name)

        if refit:
            model.fit(X, y)
        
        score = cross_val_score(model, X, y, cv = cv, scoring = scoring, n_jobs= -1)

        min_score.append(np.min(score))
        max_score.append(np.max(score))
        mean_score.append(np.mean(score))
        scores.append(score)
        t1 = time.time()
        
        if verbose:
            print(f"Iteration: {i} done in {round((t1-t0)/60,2)} minutes")
            print(f"Mean score for model: {names[i]}: {mean_score[i]}")
        
            
    
    frame_summary = pd.DataFrame({'Min':min_score, 'Average': mean_score, 'Max': max_score,}, index = names).sort_values(by = 'Average')

    frame_scores = pd.DataFrame(np.vstack(scores).T, columns = names) 


    fig, ax  = plt.subplots(1,2, figsize = (15,7))

    frame_summary.plot.barh(edgecolor = 'black', ax = ax[0], cmap = 'RdYlBu')
    ax[0].legend(loc = 'best')
    ax[0].set_xlabel("Score")

    frame_scores.boxplot(ax = ax[1])
    ax[1].set_title("Model scores distribution")
    ax[1].set_ylabel("Score")
    ax[1].tick_params(labelrotation=90)

In [ ]:
def plot_importances(estimator, X, y, scoring = None, n_repeats = 5, n_jobs = -1):
    """
    Computes permutation feature importance for a given model
    """
    pimp = permutation_importance(estimator= estimator, X= X, y = y, n_repeats= n_repeats, n_jobs = n_jobs)
    
    df = pd.DataFrame({"Mean performance decrease":pimp.importances_mean}, index = X.columns).sort_values(by = "Mean performance decrease")
    
    fig, ax = plt.subplots(figsize = (10,5))
    
    df.plot.barh(ax = ax, edgecolor = "black", cmap = "RdYlBu")
    ax.set_title("Importances")

    

The goal of this homework is to provide a realistic setting for a machine learning task.
Therefore instructions will not specify the exact steps to carry out. Instead, it is part of the
assignment to identify promising features, models and preprocessing methods and apply them
as appropriate.
**The overall goal is to predict the price of a used vehicle on craigslist**

### Task 1 Identify Features
Assemble a dataset consisting of features and target (for example in a dataframe or in two
arrays X and y). What features are relevant for the prediction task?
Are there any features that should be excluded because they leak the target information?
Show visualizations or statistics to support your selection.
You are not required to use the description column, but you can try to come up with relevant
features using it. Please don’t use bag-of-word approaches for now as we’ll discuss these later
in the class.

In [ ]:
data = pd.read_csv("/kaggle/input/craigslist-carstrucks-data/vehicles.csv")

In [ ]:
# data = data.sample(frac = .5, random_state = 1990)

In [ ]:
data.sample(7)

In [ ]:
data.info()

In [ ]:
data.describe().T

In [ ]:
data = data[(data.price >= 549) & (data.year > 0)].reset_index(drop = True)

In [ ]:
data.describe().T

In [ ]:
# Thanks to : https://www.kaggle.com/aantonova/some-new-risk-and-clusters-features
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df


In [ ]:
data = reduce_mem_usage(data)

In [ ]:
data.info()

In [ ]:
columns_to_drop = ["id", "url", "region_url", "vin", "image_url", "description", "county"]

In [ ]:
data["train_test"] = np.random.choice(a = ["train", "test"], p = [.70, .30], size = data.shape[0])

In [ ]:
Xy_train = data[data.train_test == "train"].drop(columns_to_drop, axis = 1)
Xy_test = data[data.train_test == "test"].drop(columns_to_drop, axis = 1)

## EDA

In [ ]:
Xy_train.price.describe(percentiles = np.arange(0,1,.01))[4:].plot(marker = "x", color = "darkgreen")
plt.title("Precentiles plot for price");

In [ ]:
Xy_train.odometer.describe(percentiles = np.arange(0,1,.01))[4:].plot(marker = "x", color = "darkgreen")
plt.title("Precentiles plot for odometer");

In [ ]:
data.describe().T

In [ ]:
# Remove outliers in target but only using as reference the values on the train set
Xy_train = Xy_train[Xy_train.price <= np.quantile(Xy_train.price,.99)].reset_index(drop = True)
Xy_test = Xy_test[Xy_test.price <= np.quantile(Xy_train.price,.99)].reset_index(drop = True)

# Remove outliers in target but only using as reference the values on the train set
Xy_train = Xy_train[Xy_train.odometer <= np.quantile(Xy_train.odometer.fillna(0),.99)].reset_index(drop = True)
Xy_test = Xy_test[Xy_test.odometer <= np.quantile(Xy_train.odometer.fillna(0),.99)].reset_index(drop = True)

In [ ]:
Xy_train.groupby("manufacturer").region.count().sort_values().plot.bar(figsize = (17,7),
                                                           alpha = .5,
                                                           color = "darkred",
                                                           edgecolor = "black")
plt.tight_layout()
plt.title("Sold cars by manufacturer");

In [ ]:
Xy_train.groupby("year").region.count().to_frame().plot.bar(figsize = (17,7),
                                                           alpha = .5,
                                                           color = "grey",
                                                           edgecolor = "black")
plt.tight_layout()
plt.title("Sold cars by model year");

In [ ]:
Xy_train.groupby("year").price.mean().to_frame().plot.bar(figsize = (17,7),
                                                           alpha = .5,
                                                           color = "darkred",
                                                           edgecolor = "black")
plt.tight_layout()
plt.title("Mean price by model year");

In [ ]:
# Group unreesentative groups of manufacturer and year
lux = ["ferrari", "porche", "aston-martin", "land rover", "datsun", "alfa-romeo", "tesla", "harley-davidson", "morgan"]
key = ["rare"]*9
dict_manu = dict(zip(lux,key))
Xy_train.manufacturer.replace(dict_manu, inplace = True)

# Group unreesentative groups of manufacturer and year
Xy_train["year"] = np.where(Xy_train.year < 1960, 1900, Xy_train.year)

Xy_test.manufacturer.replace(dict_manu, inplace = True)

# Group unreesentative groups of manufacturer and year
Xy_test["year"] = np.where(Xy_test.year < 1960, 1900, Xy_test.year)

In [ ]:
fig, ax = plt.subplots(1,2)
Xy_train.groupby("year").region.count().to_frame().plot.bar(figsize = (30,7),
                                                           alpha = .5,
                                                           color = "grey",
                                                           edgecolor = "black", ax = ax[0])

ax[0].set_title("Sold cars by model year")


Xy_train.groupby("manufacturer").region.count().sort_values().plot.bar(figsize = (30,7),
                                                           alpha = .5,
                                                           color = "darkred",
                                                           edgecolor = "black", ax = ax[1])

ax[1].set_title("Sold cars by manufacturer")
plt.tight_layout();

In [ ]:
fig, ax = plt.subplots(1,2,figsize = (12,7))
Xy_train.groupby("region").agg({"price":np.mean}).sort_values(by = "price").head(10).plot.barh(edgecolor = "black", ax = ax[0])
ax[0].set_title("Top 10 Regions with the lowest avg prices")

Xy_train.groupby("region").agg({"price":np.mean}).sort_values(by = "price").tail(10).plot.barh(color = "orange",edgecolor = "black", ax = ax[1])
ax[1].set_title("Top 10 Regions with the highest avg prices")
plt.tight_layout();


In [ ]:
fig, ax = plt.subplots(1,2,figsize = (12,7))
Xy_train.groupby("state").agg({"price":np.mean}).sort_values(by = "price").head(10).plot.barh(edgecolor = "black", ax = ax[0])
ax[0].set_title("Top 10 States with the lowest avg prices")

Xy_train.groupby("state").agg({"price":np.mean}).sort_values(by = "price").tail(10).plot.barh(color = "orange",edgecolor = "black", ax = ax[1])
ax[1].set_title("Top 10 States with the highest avg prices")
plt.tight_layout();

In [ ]:
Xy_train.groupby("manufacturer").agg({"price":np.mean}).sort_values(by = "price").plot.barh(figsize = (12,7),color = "orange",edgecolor = "black")
plt.title("Average price by manufacturer")
plt.tight_layout();

In [ ]:
Xy_train.groupby("condition").agg({"price":np.mean}).sort_values(by = "price").plot.barh(figsize = (12,7),
                                                                                         color = "darkgreen",edgecolor = "black")
plt.title("Average price by condition")
plt.tight_layout();

In [ ]:
plt.scatter(np.log1p(Xy_train.odometer), np.log1p(Xy_train.price), alpha = .2)
plt.title("Odometer vs price")
plt.xlabel("Log odometer")
plt.ylabel("Log price");

### Task 2 Preprocessing and Baseline Model
Create a simple minimum viable model by doing an initial selection of features, doing
appropriate preprocessing and cross-validating a linear model. Feel free to exclude features or
do simplified preprocessing for this task. As mentioned before, you don’t need to validate the
model on the whole dataset.

In [ ]:
X_train = Xy_train.drop(["price", "lat", "long", "train_test"], axis = 1).reset_index(drop = True)
y_train = Xy_train["price"]

X_test = Xy_test.drop(["price", "lat", "long", "train_test"], axis = 1).reset_index(drop = True)
y_test = Xy_test["price"]

In [ ]:
cat_feat = X_train.select_dtypes(include = "object").columns.tolist()
cont_feat = X_train.select_dtypes(exclude = "object").columns.tolist()

numeric_transformer = Pipeline([("imputer", SimpleImputer(strategy= "median")),
                                ("binning", KBinsDiscretizer(encode = "onehot-dense", strategy= "kmeans"))])

obj_transformer = Pipeline([("Imputer", SimpleImputer(strategy= "constant", fill_value = "missing")),
                                ("targenc", OneHotEncoder(handle_unknown = "ignore"))])

scaler = make_column_transformer((numeric_transformer, ["odometer"]), (obj_transformer, ["manufacturer", "condition", "cylinders"]))

In [ ]:
pipe_Lasso = Pipeline([("preprocessor", scaler), ("model", Lasso())])
pipe_Ridge = Pipeline([("preprocessor", scaler), ("model", Ridge())])
pipe_OLS = Pipeline([("preprocessor", scaler), ("model", SGDRegressor(penalty = "elasticnet"))])
pipe_SVM = Pipeline([("preprocessor", scaler), ("model", LinearSVR())])


models = [pipe_Lasso, pipe_Ridge, pipe_OLS, pipe_SVM]


In [ ]:
plot_cv_score(models_list = models,X = X_train[['odometer', 'manufacturer', 'condition', 'cylinders']] ,y = y_train, refit = True, cv = 5)

In [ ]:
visualizer = ResidualsPlot(pipe_OLS)
visualizer.fit(X_train[['odometer', 'manufacturer', 'condition', 'cylinders']] ,y = y_train)
visualizer.score(X_test, y_test)
visualizer.show();

Linear models are giving negative predictions in some cases, it might be due to the non linear functional form as hipotesys of OLS model as presented [here](https://stats.stackexchange.com/questions/145383/getting-negative-predicted-values-after-linear-regression)

So I'm going to use a tranformation for the targe variable (Price)

In [ ]:
pipe_Lasso = Pipeline([("preprocessor", scaler), ("model", TransformedTargetRegressor(regressor = Lasso(), func = np.log1p, inverse_func = np.expm1))])
pipe_Ridge = Pipeline([("preprocessor", scaler), ("model", TransformedTargetRegressor(regressor = Ridge(), func = np.log1p, inverse_func = np.expm1))])
pipe_OLS = Pipeline([("preprocessor", scaler), ("model", TransformedTargetRegressor(regressor = SGDRegressor(penalty = "elasticnet"), func = np.log1p, inverse_func = np.expm1))])
pipe_SVM = Pipeline([("preprocessor", scaler), ("model", TransformedTargetRegressor(regressor = LinearSVR(), func = np.log1p, inverse_func = np.expm1))])


models = [pipe_Lasso, pipe_Ridge, pipe_OLS, pipe_SVM]


In [ ]:
plot_cv_score(models_list = models,X = X_train[['odometer', 'manufacturer', 'condition', 'cylinders']] ,y = y_train, refit = True, cv = 5)

In [ ]:
visualizer = ResidualsPlot(pipe_SVM)
visualizer.fit(X_train[['odometer', 'manufacturer', 'condition', 'cylinders']] ,y = y_train)
visualizer.score(X_test, y_test)
visualizer.show();

### Task 3 Feature Engineering
Create derived features and perform more in-depth preprocessing and data cleaning. Does this
improve your model? In particular, think about how to encode categorical variables and
whether adding interactions (for example using PolynomialFeatures or manually) might help.

In [ ]:
# Based on descriptive statistics I'm going to drop useless variables, also I'm dropping variables with quite much categories that might become aso sparce when encoding
X_train = Xy_train.drop(["price", "train_test", "state", "region", "model", "paint_color"], axis = 1).reset_index(drop = True)
y_train = Xy_train["price"]

X_test = Xy_test.drop(["price", "train_test", "state", "region", "model", "paint_color"], axis = 1).reset_index(drop = True)
y_test = Xy_test["price"]

In [ ]:
X_train.info()

In [ ]:
cat_feat = X_train.select_dtypes(include = "object").columns.tolist()
cont_feat = X_train.select_dtypes(exclude = "object").columns.tolist()

numeric_transformer = Pipeline([("imputer", SimpleImputer(strategy = "constant", fill_value = 0)),
                                ("binning", KBinsDiscretizer(encode = "onehot-dense", strategy= "kmeans", n_bins = 5))])

obj_transformer = Pipeline([("Imputer", SimpleImputer(strategy= "constant", fill_value = "missing")),
                                ("onehot", OneHotEncoder(handle_unknown = "ignore", ))])

scaler = make_column_transformer((numeric_transformer, cont_feat), (obj_transformer, cat_feat))

In [ ]:
X_train.describe().T

In [ ]:
pipe_Lasso = Pipeline([("preprocessor", scaler), ("model", TransformedTargetRegressor(regressor = Lasso(), func = np.log1p, inverse_func = np.expm1))])
pipe_Ridge = Pipeline([("preprocessor", scaler), ("model", TransformedTargetRegressor(regressor = Ridge(), func = np.log1p, inverse_func = np.expm1))])
pipe_OLS = Pipeline([("preprocessor", scaler), ("model", TransformedTargetRegressor(regressor = SGDRegressor(penalty = "elasticnet"), func = np.log1p, inverse_func = np.expm1))])
pipe_SVM = Pipeline([("preprocessor", scaler), ("model", TransformedTargetRegressor(regressor = LinearSVR(), func = np.log1p, inverse_func = np.expm1))])

models = [pipe_Lasso, pipe_Ridge, pipe_OLS, pipe_SVM]

In [ ]:
plot_cv_score(models_list = models,X = X_train ,y = y_train, refit = True, cv = 5)

In [ ]:
plot_importances(pipe_Ridge, X_train, y_train, scoring = None, n_jobs = -1)

### Task 4 Any model
Use any regression model we discussed (trees, forests, gradient boosting, SVM) to improve
your result. You can (and probably should) change your preprocessing and feature engineering
to be suitable for the model. You are not required to try all of these models. Tune parameters
as appropriate.

In [ ]:
cat_feat = X_train.select_dtypes(include = "object").columns.tolist()
cont_feat = X_train.select_dtypes(exclude = "object").columns.tolist()

numeric_transformer = Pipeline([("imputer", SimpleImputer(strategy = "constant", fill_value = 0))])

obj_transformer = Pipeline([("Imputer", SimpleImputer(strategy= "constant", fill_value = "missing")),
                                ("onehot", OneHotEncoder(handle_unknown = "ignore"))])

scaler = make_column_transformer((numeric_transformer, cont_feat), (obj_transformer, cat_feat))

# pipe_Ensem = Pipeline([("scaler", scaler), ("model", BaggingRegressor(base_estimator = XGBRFRegressor(), n_estimators = 50))])
pipe_Xgb = Pipeline([("scaler", scaler), ("model", XGBRFRegressor())])


models = [pipe_Xgb]

In [ ]:
plot_cv_score(models_list = models,X = X_train ,y = y_train, refit = True, cv = 5)

In [ ]:
plot_importances(pipe_Xgb, X_train, y_train)

In [ ]:
visualizer = ResidualsPlot(pipe_Xgb)
visualizer.fit(X_train ,y = y_train)
visualizer.score(X_test, y_test)
visualizer.show();

### Task 5 Feature Selections
Identify features that are important for your best model. Which features are most influential,
and which features could be removed without decrease in performance? Does removing
irrelevant features make your model better? (This will be discussed in the lecture on 03/04).


In [ ]:
X_train = X_train.drop(["long", "lat", "title_status", "size", "transmission"], axis = 1)
X_test = X_test.drop(["long", "lat", "title_status", "size", "transmission"], axis = 1)

In [ ]:
cat_feat = X_train.select_dtypes(include = "object").columns.tolist()
cont_feat = X_train.select_dtypes(exclude = "object").columns.tolist()

numeric_transformer = Pipeline([("imputer", SimpleImputer(strategy = "constant", fill_value = 0))])

obj_transformer = Pipeline([("Imputer", SimpleImputer(strategy= "constant", fill_value = "missing")),
                                ("onehot", OneHotEncoder(handle_unknown = "ignore"))])

scaler = make_column_transformer((numeric_transformer, cont_feat), (obj_transformer, cat_feat))

pipe_EnsemRF = Pipeline([("scaler", scaler), ("model", BaggingRegressor(base_estimator = XGBRFRegressor(), n_estimators = 5))])
pipe_XgbRF = Pipeline([("scaler", scaler), ("model", XGBRFRegressor())])
pipe_Xgb = Pipeline([("scaler", scaler), ("model", XGBRegressor())])
pipe_EnsemBoost = Pipeline([("scaler", scaler), ("model", BaggingRegressor(base_estimator = XGBRegressor(), n_estimators = 5))])


models = [pipe_EnsemRF, pipe_XgbRF,pipe_Xgb,pipe_EnsemBoost ]

In [ ]:
plot_cv_score(models_list = models,X = X_train ,y = y_train, refit = True, cv = 5)

In [ ]:
plot_importances(pipe_Xgb, X_train, y_train)

In [ ]:
visualizer = ResidualsPlot(pipe_Xgb)
visualizer.fit(X_train ,y = y_train)
visualizer.score(X_test, y_test)
visualizer.show();

This is having the undesired behaviuor of predicting negative values for this: [solution](https://github.com/dmlc/xgboost/issues/1581)

### Task 6 An explainable model
Can you create an “explainable” model that is nearly as good as your best model?
An explainable model should be small enough to be easily inspected - say a linear model with
few enough coefficients that you can reasonably look at all of them, or a tree with a small
number of leaves etc.**

In [ ]:
# Instead I'm going to use a model agnostic technique to be able to make my best model "explainable"
from sklearn.inspection import plot_partial_dependence

In [ ]:
relevant_feat = ["year", "odometer"]
plot_partial_dependence(estimator=pipe_EnsemBoost, X =X_train, features =relevant_feat, n_jobs = -1, verbose = 2);